In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

from torch.distributions.log_normal import LogNormal

# Internal packages
from data_loader import KidneyStoneDataset, ToTensor
from model import binary_ks_net, binary_neg_loglik, cont_rec_ks_net, cont_rec_neg_loglik, cont_size_ks_net, cont_size_neg_loglik
from train import train

## Gamma generative process with log-normal parametrization

In [2]:
# Hyperparameters
BATCH_SIZE = 128
EPOCHS     = 150
LEARN_R    = 1e-2 #RMS
#LEARN_R    = 1e-3 #SGD
N_HU       = 4
#NLA = F.relu
NLA = nn.LeakyReLU(1) # For a linear neural network

# Initialize the dataset
data = KidneyStoneDataset("./data/ks_cont_size_data_g.npy", transform=ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE)

# Initialize the model
model = cont_size_ks_net(N_HU, NLA)

# Optimizers
#optimizer = optim.SGD(model.parameters(), lr=LEARN_R, weight_decay=0.002)
#optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R, weight_decay=0.002)
optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R)

In [3]:
cum_loss = train(model, optimizer, cont_size_neg_loglik, train_loader, EPOCHS)

In [4]:
# First, we get the parameters of the size variable:
n_samples = 50
arbitrary_query = torch.tensor([1., 1., 1.]) # It is only important that the first element is 1
mu_L, log_sigma_L, _, _, _ = model(arbitrary_query.unsqueeze(0))
sigma_L = torch.exp(log_sigma_L)
L_dist = LogNormal(mu_L, sigma_L)
L_samples = L_dist.sample((n_samples,)).view(n_samples,1)

In [5]:
t_1 = torch.cat((L_samples, torch.ones(n_samples, 2)), 1)
t_0 = torch.cat((L_samples, torch.zeros(n_samples, 1) ,torch.ones(n_samples, 1)), 1)

In [6]:
# Probabilities with Treatment A
_, _, _, mu_t1, _ = model(t_1) # Probabilities with KS = L
_, _, _, mu_t0, _ = model(t_0) # Probabilities with KS = L

In [7]:
# By monte carlo integration, we are getting mu from the value of the variables from the distribution
for n in [1, 5, 25, 50]:
    print("%d: %f" % (n, torch.mean(mu_t1[:n])-torch.mean(mu_t0[:n])))

1: 3.775667
5: 3.775668
25: 3.775667
50: 3.775668


## Log-normal generative process with log-normal parametrization

In [10]:
# Hyperparameters
BATCH_SIZE = 128
EPOCHS     = 150
LEARN_R    = 1e-2
N_HU       = 4
#NLA = F.relu
NLA = nn.LeakyReLU(1) # For a linear neural network

# Initialize the dataset
data = KidneyStoneDataset("./data/ks_cont_size_data_ln.npy", transform=ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE)

# Initialize the model
model = cont_size_ks_net(N_HU, NLA)

# Optimizers
#optimizer = optim.SGD(model.parameters(), lr=LEARN_R, weight_decay=0.1)
optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R)

In [11]:
cum_loss = train(model, optimizer, cont_size_neg_loglik, train_loader, EPOCHS)

In [12]:
# First, we get the parameters of the size variable:
n_samples = 50
arbitrary_query = torch.tensor([1., 1., 1.]) # It is only important that the first element is 1
mu_L, log_sigma_L, _, _, _ = model(arbitrary_query.unsqueeze(0))
sigma_L = torch.exp(log_sigma_L)
L_dist = LogNormal(mu_L, sigma_L)
L_samples = L_dist.sample((n_samples,)).view(n_samples,1)

In [13]:
t_1 = torch.cat((L_samples, torch.ones(n_samples, 2)), 1)
t_0 = torch.cat((L_samples, torch.zeros(n_samples, 1) ,torch.ones(n_samples, 1)), 1)

In [14]:
# Probabilities with Treatment A
_, _, _, mu_t1, _ = model(t_1) # Probabilities with KS = L
_, _, _, mu_t0, _ = model(t_0) # Probabilities with KS = L

In [15]:
# By monte carlo integration, we are getting mu from the value of the variables from the distribution
for n in [1, 5, 25, 50]:
    print("%d: %f" % (n, torch.mean(mu_t1[:n])-torch.mean(mu_t0[:n])))

1: 4.091660
5: 4.091659
25: 4.091660
50: 4.091658
